In [1]:
from statsmodels.tsa.stattools import acf, pacf # computación de las correlaciones
from statsmodels.tsa.arima.model import ARIMA   # estimación de modelos ARIMA
from matplotlib.lines import Line2D             # para ajustar las leyendas en los plots
from scipy.signal import correlate              # computación de las correlaciones entre señales
import matplotlib.pyplot as plt                 # visualización de datos
import tensorflow as tf                         # modelos de neuronas artificiales
from scipy import stats                         # para estadística
import pandas as pd                             # tratar con DataFrames
import numpy as np                              # cálculo sobre matrices
import itertools                                # combinatoria
import datetime                                 # para parsear las fechas
import tqdm                                     # para los bucles

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv('data/[2021-06-03] - state.csv')               # lectura del csv
df.time = pd.to_datetime(df['time'], unit='s', origin='unix')   # parseamos las fechas
df = df.drop(columns = ["time", "id", "sensor_id", "state_time"])       # droppeamos las columnas sin información relevante
df.state = (df.state == "On").astype(int)
df.head()

In [ ]:
df = df.iloc[36000:48000]
df.head()

In [ ]:
df.radon.plot()

In [ ]:
def generate_windows(data, x_len, y_len):
    """
    Construye una matriz de ventanas dados los siguientes argumentos:
    + data: list or array
    + x_len : longitud de la ventana que se usará para predecir
    + y_len: tamaño del horizonte
    """
    N = len(data) - (x_len + y_len) + 1
    _data = np.zeros((N, (x_len + y_len)))
    
    for i in range(N):
        _data[i,:] = data[i:(i+x_len+y_len)]
    return _data

In [ ]:
def mix_data(data):
    """Devuelve la matriz formateada"""
    complete = np.empty((list(data["radon"].shape) + [len(data.keys())]))
    keys = list(data.keys())
    for i in range(len(keys)):
        complete[:,:,i] = data[keys[i]]
    return complete

In [ ]:
df_normalizado = (df - df.min()) / (df.max() - df.min())
df_normalizado.head()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor              = 'val_loss',
    patience             = 30,
    restore_best_weights = True
)
porcentaje_train = 0.9

resultados = pd.DataFrame(
    {i: [None]*20 for i in [25]}#[1, 5, 10, 15]}
)

cnt = 0

for number_of_covariates in range(4):
    for covariate_combination in itertools.combinations(("state", "humidity", "pressure", "tvoc"), 
                                                        number_of_covariates):
        print(f"EXECUTING OVER: {covariate_combination}".center(100, "*"))
        # ejecutamos para cada ventana
        for window_size in [25]: #[1, 5, 10, 15]:
            print("\n")
            print(f"EXECUTING {window_size} WINDOW SIZE".center(100), "-")
            print("\n")
            ventanas = {i: generate_windows(df_normalizado[i], window_size, 1)
                        for i in ["radon"] + list(covariate_combination) }
            print("Ventanas computadas, construyendo modelo")
            
            data = mix_data(ventanas)
            N_total          = data.shape[0]
            test             = data[int(porcentaje_train*N_total):, :, :]
            data             = data[:int(porcentaje_train*N_total), :, :]
            
            model = tf.keras.models.Sequential(
                [
                    tf.keras.layers.InputLayer((window_size, len(ventanas.keys()))),
                    tf.keras.layers.LSTM(window_size * 2),
                    tf.keras.layers.Dense(16, activation = "relu"),
                    tf.keras.layers.Dense(16, activation = "relu"),
                    tf.keras.layers.Dense(1)
                ],
            )
            
            model.compile(
                loss = tf.keras.losses.MeanSquaredError(),
                metrics = tf.keras.metrics.RootMeanSquaredError(),
                optimizer = tf.keras.optimizers.RMSprop()
            )
            
            model.summary()
            
            model.fit(
                x                = data[:, :window_size, :],
                y                = data[:, window_size:, 0],
                epochs           = 1_000,
                shuffle          = False,
                validation_split = 0.2, 
                callbacks        = [callback,],
                verbose          = 0,
            )
            
            # Modelo entrenado.
            
            # vamos a realizar las predicciones y reescalarlo
            prediccion = model.predict(test[:, :window_size, :]) * (df.max().radon - df.min().radon) + df.min().radon
            real       = test[:, window_size:, 0] * (df.max().radon - df.min().radon) + df.min().radon
            
            rmse_test = ( sum((prediccion - real) ** 2) / len(prediccion) ) ** (1/2)
            resultados[window_size][cnt] = rmse_test 
            print(f"Window {window_size}, covariates {covariate_combination}: {resultados[window_size][cnt]}")
        cnt += 1    
        print("*" * 100, "\n"*2)

In [ ]:
print(resultados[:15].to_latex())

# Ajuste fino del modelo con state: selección de neuronas

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor              = 'val_loss',
    patience             = 30,
    restore_best_weights = True
)
porcentaje_train = 0.9
window_size = 15
resultados = [None] * len([4, 8, 16, 32, 64])

ventanas = {i: generate_windows(df_normalizado[i], 15, 1)
            for i in ["radon", "state"] }
print("Ventanas computadas, construyendo modelo")

data = mix_data(ventanas)
N_total          = data.shape[0]
test             = data[int(porcentaje_train*N_total):, :, :]
data             = data[:int(porcentaje_train*N_total), :, :]

for idx, n_hidden_units in enumerate([4, 8, 16, 32, 64]):
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.InputLayer((15, 2)),
            tf.keras.layers.LSTM(n_hidden_units),
            tf.keras.layers.Dense(32, activation = "relu"),
            tf.keras.layers.Dense(32, activation = "relu"),
            tf.keras.layers.Dense(1)
        ],
    )

    model.compile(
        loss = tf.keras.losses.MeanSquaredError(),
        metrics = tf.keras.metrics.RootMeanSquaredError(),
        optimizer = tf.keras.optimizers.RMSprop()
    )

    model.summary()

    model.fit(
        x                = data[:, :15, :],
        y                = data[:, 15:, 0],
        epochs           = 1_000,
        shuffle          = False,
        validation_split = 0.2, 
        callbacks        = [callback,],
        verbose          = 0,
    )

    # Modelo entrenado.

    # vamos a realizar las predicciones y reescalarlo
    prediccion = model.predict(test[:, :15, :]) * (df.max().radon - df.min().radon) + df.min().radon
    real       = test[:, 15:, 0] * (df.max().radon - df.min().radon) + df.min().radon

    rmse_test = ( sum((prediccion - real) ** 2) / len(prediccion) ) ** (1/2)
    print(f"RMSE con {n_hidden_units} unidades ocultas: {rmse_test}")
    resultados[idx] = rmse_test 

## Ajuste final del modelo

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor              = 'val_loss',
    patience             = 30,
    restore_best_weights = True
)
porcentaje_train = 0.9
window_size = 15

ventanas = {i: generate_windows(df_normalizado_[i], 15, 1)
            for i in ["radon", "state"] }
print("Ventanas computadas, construyendo modelo")

data = mix_data(ventanas)
N_total          = data.shape[0]
test             = data[int(porcentaje_train*N_total):, :, :]
data             = data[:int(porcentaje_train*N_total), :, :]

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.InputLayer((15, 2)),
        tf.keras.layers.LSTM(4),
        tf.keras.layers.Dense(32, activation = "relu"),
        tf.keras.layers.Dense(1)
    ],
)

model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    metrics = tf.keras.metrics.RootMeanSquaredError(),
    optimizer = tf.keras.optimizers.RMSprop()
)

model.summary()

model.fit(
    x                = data[:, :15, :],
    y                = data[:, 15:, 0],
    epochs           = 1_000,
    shuffle          = False,
    validation_split = 0.2, 
    callbacks        = [callback,],
    verbose          = 0,
)

# Modelo entrenado.

# vamos a realizar las predicciones y reescalarlo
prediccion = model.predict(test[:, :15, :]) * (df.max().radon - df.min().radon) + df.min().radon
real       = test[:, 15:, 0] * (df.max().radon - df.min().radon) + df.min().radon

rmse_test = ( sum((prediccion - real) ** 2) / len(prediccion) ) ** (1/2)
print(f"RMSE con {n_hidden_units} unidades ocultas: {rmse_test}")

In [ ]:
reconstruccion = pd.DataFrame(
    {
        "pred": [i[0] for i in prediccion],
        "real": [i[0] for i in real]
    }
)

In [ ]:
reconstruccion.head()

In [ ]:
abs(reconstruccion.pred - reconstruccion.real).mean()

In [ ]:
reconstruccion.to_csv("data/reconstruccion_lstm_bueno.csv", index = False)

## precisión en otra época del año

In [ ]:
df_ = pd.read_csv('data/[2021-06-03] - state.csv')               # lectura del csv
df_.time = pd.to_datetime(df_['time'], unit='s', origin='unix')   # parseamos las fechas
df_ = df_.drop(columns = ["time", "id", "sensor_id", "state_time"])       # droppeamos las columnas sin información relevante
df_.state = (df_.state == "On").astype(int)

In [ ]:
df_ = df_.iloc[82400:82800]
df_.head()

In [ ]:
df_normalizado_ = (df_ - df.min()) / (df.max() - df.min())
df_normalizado_.head()

In [ ]:
ventanas = {i: generate_windows(df_normalizado_[i], 15, 1)
            for i in ["radon", "state"] }

data = mix_data(ventanas)
N_total          = data.shape[0]
test             = data[int(porcentaje_train*N_total):, :, :]
data             = data[:int(porcentaje_train*N_total), :, :]

In [ ]:
prediccion = model.predict(data[:, :15, :]) * (df.max().radon - df.min().radon) + df.min().radon
real       = data[:, 15:, 0] * (df.max().radon - df.min().radon) + df.min().radon

In [ ]:
( sum((prediccion - real) ** 2) / len(prediccion) ) ** (1/2)

In [ ]:
reconstruccion = pd.DataFrame(
    {
        "pred": [i[0] for i in prediccion],
        "real": [i[0] for i in real]
    }
)

In [ ]:
reconstruccion.to_csv("data/reconstruccion_lstm_otra_epoca.csv", index = False)

In [ ]:
(reconstruccion.pred - reconstruccion.real).plot()

In [ ]:
reconstruccion.plot()